Reference : https://juliabook.chkwon.net/book/lagrangian

# Introduction

* For Mixed Integer Liner Programming Problems and Combinatorial Optimization Problems
* We relax hard constraints to make the relaxed problems relatively eaiser to solve
* While we solve these easy problems multiple times, we update the values of Lagrangian multipliers adequately

## Lower and Upper Bounds

* Z_LB and Z_UB
* Optimality Gap
* The relaxed Lagrangian problem provides LB.
* We want the Lb are close to the optimal Z* -> We need to make a good choice of lambda.

* Linear Programming relaxation also provides an lower bound which is usually not very good.

* To find an upper bound, we just need to find a feasible solution to the original problem. -> Use the solution to the Lagrangian problem to find a good upper bound.

## Subgradient Optimization

* When updating lambdas, we basically want to maximize the lower bound.
* To solve this problem, we use a subgradient optimization method.

## Summary

1. Which constraints to be relaxed
2. How to obtain a feasible solution -> upper bound
3. how to update the Lagrangian multiplier -> lower bound

## Example : The p-Median Problem

* Facility location problem
* NP-hard

In [1]:
using DelimitedFiles
using JuMP, GLPK
using PyPlot

In [2]:
# Solving the p-median problem by Lagrangian Relaxation
p=3

3

### 1. Reading the Data File

In [3]:
# Reading demand data
d, header = readdlm("ch10_demand.csv", ',', header=true)

# Reading transportation cost data
data = readdlm("ch10_cost.csv", ',')
cc = data[2:end, 2:end]
c = convert(Array{Float64,2}, cc)

7×10 Matrix{Float64}:
 10.0   7.0  11.0  12.0  32.0  15.0  20.0  26.0   4.0  41.0
 13.0  17.0  31.0  37.0  21.0   5.0  13.0  15.0  14.0  12.0
  4.0  13.0  14.0  22.0   8.0  31.0  26.0  11.0  12.0  23.0
 21.0  21.0  13.0  18.0   9.0  27.0  11.0  16.0  26.0  32.0
 32.0  18.0  11.0  14.0  11.0  11.0  16.0  32.0  34.0   8.0
 15.0   9.0  13.0  12.0  14.0  15.0  32.0   8.0  12.0   9.0
 28.0  32.0  15.0   2.0  17.0  12.0   9.0   6.0  11.0   6.0

In [4]:
# the length of 'd' and the number of columns in 'c' must match
@assert length(d) == size(c,2) # If the given statement is true, it just does nothing and proceeds to the next line of the code

* 10 customers and 7 candidate locations

In [5]:
locations = 1:size(c,1) # the set, I = {1,2,...,7}
customers = 1:length(d) # the set, J = {1,2,...,10}

# making these data global so that any function can access data
global locations, customers, c

### 2. Solving the p-Median Problem Optimally

In [6]:
function optimal(p)
    m = Model(GLPK.Optimizer)

    @variable(m, x[i in locations, j in customers] >= 0)
    @variable(m, y[i in locations], Bin)

    @objective(m, Min, sum(d[j]*c[i,j]*x[i,j] for i in locations, j in customers))

    @constraint(m, [j in customers], sum(x[i,j] for i in locations) == 1)
    @constraint(m, sum(y[i] for i in locations) == p)
    @constraint(m, [i in locations, j in customers], x[i,j] <= y[i])

    optimize!(m)

    Z_opt = objective_value(m)
    x_opt = value.(x)
    y_opt = value.(y)

    return Z_opt, x_opt, y_opt
end

optimal (generic function with 1 method)

### 3. Lagrangian Relaxation

To apply the Lagrangian relaxation method, we relax the first constraint that enforces the total assignment for each demand is 100%.

### 4.Finding the Lower bounds

In [14]:
function lower_bound(lambda)
    # Step 1: Computing v
    v = Array{Float64}(undef, size(locations))
    for i in locations
      v[i] = 0
      for j in customers
        v[i] = v[i] + min(0, d[j]*c[i,j] - lambda[j] )
      end
    end
  
    # Step 2: Sorting v from the most negative to zero
    idx = sortperm(v)
  
    # Step 3: Determine y
    y = zeros(Int, size(locations))
    y[idx[1:p]] .= 1
  
    # Step 4: Determine x
    x = zeros(Int, length(locations), length(customers))
    for i in locations, j in customers
      if y[i]==1 && d[j]*c[i,j]-lambda[j]<0
        x[i,j] = 1
      end
    end
  
    # Computing the Z_D(lambda^k)
    Z_D = 0.0
    for j in customers
      Z_D = Z_D + lambda[j]
      for i in locations
        Z_D = Z_D + d[j]*c[i,j]*x[i,j] - lambda[j]*x[i,j]
      end
    end
  
    return Z_D, x, y
  end

lower_bound (generic function with 1 method)

### 5. Finding Upper Bounds

In [8]:
function upper_bound(y)
    # Computing x, given y
    x = zeros(Int, length(locations), length(customers))
    for j in customers
      idx = argmin( c[:,j] + (1 .- y) .* maximum(c) )
      x[idx,j] = 1
    end
  
    # Computing Z
    Z = 0.0
    for i in locations
      for j in customers
        Z = Z + d[j]*c[i,j]*x[i,j]
      end
    end
  
    return Z, x
  end

upper_bound (generic function with 1 method)

### 6. Updating the Lagrangian Multiplier

Subgradient optimization method

In [15]:
function lagrangian_relaxation(p)
    # The maximum number of iterations allowed
    MAX_ITER = 10000

    # To track the upper and lower bounds
    UB = Array{Float64}(undef, 0)
    LB = Array{Float64}(undef, 0)

    # The best-known upper and lower bounds
    Z_UB = Inf
    Z_LB = -Inf

    # The best-known feasible solutions
    x_best = zeros(length(locations), length(customers))
    y_best = zeros(length(locations))

    # Initial multiplier
    lambda = zeros(size(customers))

    for k=1:MAX_ITER
      # Obtaining the lower and upper bounds
      Z_D, x_D, y = lower_bound(lambda)
      Z, x = upper_bound(y)

      # Updating the upper bound
      if Z < Z_UB
        Z_UB = Z
        x_best = x
        y_best = y
      end

      # Updating the lower bound
      if Z_D > Z_LB
        Z_LB = Z_D
      end

      # Adding the bounds from the current iteration to the record
      push!(UB, Z)
      push!(LB, Z_D)

      # Determining the step size and updating the multiplier
      theta = 1.0
      residual = 1 .- transpose(sum(x_D, dims=1))
      t = theta * (Z_UB - Z_D) / sum(residual.^2)
      lambda = lambda + t * residual

      # Computing the optimality gap
      opt_gap = (Z_UB-Z_LB) / Z_UB
      if opt_gap < 0.000001
        break
      end
    end

    return Z_UB, x_best, y_best, UB, LB
end

lagrangian_relaxation (generic function with 1 method)

In [34]:
# Finding the exact optimal solution
Z_opt, x_opt, y_opt = optimal(p)

println("The optimal solution is: ", Z_opt)

The optimal solution is: 1029.0


In [35]:
# Finding a solution by Lagrangian relaxation
Z_UB, x_best, y_best, UB, LB = lagrangian_relaxation(p)
println("The best-known upper bound is: ", Z_UB)

The best-known upper bound is: 1029.0


In [13]:
iter = 1:length(LB)
fig = figure()

# Plotting two datasets
plot(iter, LB, color="red", linewidth=2.0, linestyle="-",
   marker="o", label="Lower Bound")
plot(iter, UB, color="blue", linewidth=2.0, linestyle="-.",
   marker="D", label="Upper Bound")

# Labeling axes
xlabel(L"iteration clock $k$", fontsize="xx-large")
ylabel("Bounds", fontsize="xx-large")

# Putting the legend and determining the location
legend(loc="lower right", fontsize="x-large")

# Add grid lines
grid(color="#DDDDDD", linestyle="-", linewidth=1.0)
tick_params(axis="both", which="major", labelsize="x-large")

# Title
title("Lower and Upper Bounds")
savefig("ch10_iterations.png")
#savefig("iterations.pdf")
close(fig)

![result](ch10_iterations.png)
